# A/B Testing for ShoeFly.com

In [27]:
#importing libraries
import pandas as pd

In [29]:
#read csv file and inspecting first five rows
ad_clicks = pd.read_csv('ad_clicks.csv')
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


In [30]:
#getting views per utm_source
ad_clicks_by_source = ad_clicks.groupby('utm_source').user_id.count().reset_index()
ad_clicks_by_source

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


In [31]:
#adding a column is_click which us True is ad_click_timestamp is not empty else False
ad_clicks['is_click'] = ad_clicks.ad_click_timestamp.notnull()
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


In [32]:
#grouping by utm_source and is_click to get number of people in each category
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


In [46]:
#pivoting so that enteries in is_click are columns with utm_source as index
clicks_by_source_pivot = clicks_by_source.pivot(columns = 'is_click', index = 'utm_source', values = 'user_id').reset_index()
clicks_by_source_pivot

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


In [34]:
#adding a column percent_clicked which calculates proportion of users who clicked on ads
clicks_by_source_pivot['percent_clicked'] = (clicks_by_source_pivot[True] / (clicks_by_source_pivot[True] + clicks_by_source_pivot[False])) * 100
clicks_by_source_pivot

is_click,utm_source,False,True,percent_clicked
0,email,175,80,31.372549
1,facebook,324,180,35.714286
2,google,441,239,35.147059
3,twitter,149,66,30.697674


In [47]:
#grouping by experimental_group to check how many users were shown each ad
a_or_b_show = ad_clicks.groupby('experimental_group').user_id.count().reset_index()
a_or_b_show

,experimental_group,user_id
0,A,827
1,B,827


In [49]:
#grouping by experiemtal_group and is_click to check how many users clicked each ad
a_or_b_click = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()
a_or_b_click

,experimental_group,is_click,user_id
0,A,False,517
1,A,True,310
2,B,False,572
3,B,True,255


In [50]:
#pivoting so that enteries in is_click are columns with experimental_group as index
a_or_b_click_pivot = a_or_b_click.pivot(columns = 'is_click', index = "experimental_group", values = 'user_id').reset_index()
a_or_b_click_pivot

is_click,experimental_group,False,True
0,A,517,310
1,B,572,255


In [51]:
#creating a dataframe that contains only results for ad A
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
a_clicks

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A,False
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A,False
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A,False
...,...,...,...,...,...,...
1643,fceb13ea-fd8c-446a-a61f-f977d404330a,twitter,6 - Saturday,NaN,A,False
1646,fd7d06ea-38b5-4ed9-acc9-777047db8c56,google,4 - Thursday,NaN,A,False
1647,fe570a20-448f-40ed-930b-8482b8a7c231,facebook,1 - Monday,20:7,A,True
1649,fe8b5236-78f6-4192-9da6-a76bba67cfe6,twitter,7 - Sunday,NaN,A,False


In [52]:
#creating a dataframe that contains only results for ad B
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']
b_clicks

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B,True
10,01adb2e7-f711-4ae4-a7c6-29f48457eea1,google,3 - Wednesday,NaN,B,False
...,...,...,...,...,...,...
1645,fd2a5852-f0ef-4162-84a6-107a42dc46b5,twitter,3 - Wednesday,NaN,B,False
1648,fe6cfa5a-cc63-4770-8d56-c13ac8cf5bef,google,3 - Wednesday,15:6,B,True
1650,fed3db6d-8c92-40e3-a4fb-1fb9d7337eb1,facebook,5 - Friday,NaN,B,False
1651,ff3a22ff-521c-478c-87ca-7dc7b8f34372,twitter,3 - Wednesday,NaN,B,False


In [53]:
#grouping by is_click and day to check how many users clicked on ad A
a_clicks = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
a_clicks

,is_click,day,user_id
0,False,1 - Monday,70
1,False,2 - Tuesday,76
2,False,3 - Wednesday,86
3,False,4 - Thursday,69
4,False,5 - Friday,77
5,False,6 - Saturday,73
6,False,7 - Sunday,66
7,True,1 - Monday,43
8,True,2 - Tuesday,43
9,True,3 - Wednesday,38


In [54]:
#pivoting to have enteries of is_click as columns and day as index
#a new column percent_clicked is also created to calculate proportion of users who clicked on ad A each day
a_clicks_pivot = a_clicks.pivot(columns = 'is_click', index = 'day', values = 'user_id').reset_index()
a_clicks_pivot['percent_clicked'] = (a_clicks_pivot[True] / (a_clicks_pivot[True] + a_clicks_pivot[False]))*100
a_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,70,43,38.053097
1,2 - Tuesday,76,43,36.134454
2,3 - Wednesday,86,38,30.645161
3,4 - Thursday,69,47,40.517241
4,5 - Friday,77,51,39.843750
5,6 - Saturday,73,45,38.135593
6,7 - Sunday,66,43,39.449541


In [55]:
#grouping by is_click and day to check how many users clicked on ad B
b_clicks = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
b_clicks

,is_click,day,user_id
0,False,1 - Monday,81
1,False,2 - Tuesday,74
2,False,3 - Wednesday,89
3,False,4 - Thursday,87
4,False,5 - Friday,90
5,False,6 - Saturday,76
6,False,7 - Sunday,75
7,True,1 - Monday,32
8,True,2 - Tuesday,45
9,True,3 - Wednesday,35


In [56]:
#pivoting to have enteries of is_click as columns and day as index
#a new column percent_clicked is also created to calculate proportion of users who clicked on ad B each day
b_clicks_pivot = b_clicks.pivot(columns = 'is_click', index = 'day', values = 'user_id').reset_index()
b_clicks_pivot['percent_clicked'] = (b_clicks_pivot[True] / (b_clicks_pivot[True] + b_clicks_pivot[False]))*100
b_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,81,32,28.318584
1,2 - Tuesday,74,45,37.815126
2,3 - Wednesday,89,35,28.225806
3,4 - Thursday,87,29,25.000000
4,5 - Friday,90,38,29.687500
5,6 - Saturday,76,42,35.593220
6,7 - Sunday,75,34,31.192661
